In [1]:
import pandas as pd 
import numpy as np

In [2]:
sales_df = pd.read_excel('data/train_sales.xlsx')
promo_df = pd.read_excel('data/train_promo.xlsx')

In [9]:
sales_df['DFU'].unique()

array(['Рис басмати 500 гр', 'Рис длиннозерный 500 гр',
       'Рис круглозерный 1000 гр', 'Рис круглозерный 500 гр',
       'Рис Жасмин 500 гр', 'Рис для плова 500 гр',
       'Рис длиннозерный 486 гр', 'Гречка Ядрица 1000 гр',
       'Йогурт Постный 180 гр', 'Булгур 300 гр'], dtype=object)

In [10]:
promo_df['DFU'].unique()

array(['Рис длиннозерный 500 гр', 'Рис Жасмин 500 гр',
       'Рис басмати 500 гр', 'Рис для плова 500 гр',
       'Рис круглозерный 1000 гр', 'Йогурт Постный 180 гр',
       'Рис круглозерный 500 гр', 'Булгур 300 гр',
       'Рис длиннозерный 486 гр', 'Киноа 300 гр'], dtype=object)

In [7]:
sales_df[sales_df['DFU'] == 'Рис басмати 500 гр']['Customer'].unique()

array([18, 34,  2, 29, 14,  1])

In [8]:
promo_df[promo_df['DFU'] == 'Рис басмати 500 гр']['Customer'].unique()

array([41,  6,  3, 13, 26, 15,  1, 40,  7, 45, 21, 12, 32, 24, 35, 36, 22,
       27,  8, 20, 23,  4, 43, 10, 19,  5, 30, 44])

In [13]:
for dfu_sales in sales_df['DFU'].unique():
    print(f"Кастомеры по продукту {dfu_sales}")
    print(f"train_sales: {sales_df[sales_df['DFU'] == dfu_sales]['Customer'].unique()}")
    print(f"train_promo: {promo_df[promo_df['DFU'] == dfu_sales]['Customer'].unique()}")
    print('\n')

Кастомеры по продукту Рис басмати 500 гр
train_sales: [18 34  2 29 14  1]
train_promo: [41  6  3 13 26 15  1 40  7 45 21 12 32 24 35 36 22 27  8 20 23  4 43 10
 19  5 30 44]


Кастомеры по продукту Рис длиннозерный 500 гр
train_sales: [1]
train_promo: [41 13  1 15 40 45 21 32 12  3  8 26 22 35 27  7 24  6 36 23 20  4 43  5
 19 10 44]


Кастомеры по продукту Рис круглозерный 1000 гр
train_sales: [1]
train_promo: [15 40  1]


Кастомеры по продукту Рис круглозерный 500 гр
train_sales: [1]
train_promo: [41 26 24 32 27 22  6 45 21  3 15 40 13 35  1 12  8 36  7 20 23  4 43 10
  5 19 44 30]


Кастомеры по продукту Рис Жасмин 500 гр
train_sales: [1]
train_promo: [41  6 13 26 15  1 45 21 36  3 40  7 12 22 35 32 24 27  8 20 23  4 43 10
 19  5 30 44]


Кастомеры по продукту Рис для плова 500 гр
train_sales: [1]
train_promo: [41  3 13 12 27 40 32  1 26 35 45 21 22  6 15  7  8 36 20 23  4 43  5 19
 10 30]


Кастомеры по продукту Рис длиннозерный 486 гр
train_sales: [1]
train_promo: [23 15 32 40 20 

In [3]:
sales_df.head(10)

,DFU,Customer,Period,BPV,Total Sell-in
0,Рис басмати 500 гр,18,2017-07-24,0.846000,0.846000
1,Рис басмати 500 гр,18,2017-10-23,1.692000,1.692000
2,Рис басмати 500 гр,18,2018-02-19,2.538000,2.538000
3,Рис басмати 500 гр,18,2018-04-23,1.692000,1.692000
4,Рис басмати 500 гр,18,2019-04-29,4.232100,4.232100
5,Рис басмати 500 гр,18,2019-08-26,2.257200,2.257200
6,Рис басмати 500 гр,18,2019-11-18,2.257200,2.257200
7,Рис басмати 500 гр,18,2020-02-24,2.257200,2.257200
8,Рис басмати 500 гр,18,2020-05-18,2.257200,2.257200
9,Рис басмати 500 гр,18,2020-06-08,2.254407,2.254407


In [4]:
promo_df.head(10)

,Promo,Promo №,Customer,DFU,Promo mechanic,Start Date on shelf,Promo Days on shelf,End Date on shelf,Shipment days to promo start,First Date of shipment,End Date of shipment,"Discount, %",Units SoD
0,987359,709635,41,Рис длиннозерный 500 гр,395048,2017-12-01,14.0,2017-12-14,14,2017-11-17,2018-01-18,0.15,6216.00000
1,987365,710042,13,Рис длиннозерный 500 гр,395054,2017-12-06,7.0,2017-12-12,21,2017-11-15,2017-12-17,0.20,13511.09062
2,987375,708618,1,Рис длиннозерный 500 гр,395054,2017-12-18,7.0,2017-12-24,21,2017-12-02,2017-12-24,0.25,362049.09560
3,987384,709929,15,Рис длиннозерный 500 гр,395054,2017-12-27,7.0,2018-01-02,22,2017-12-11,2018-01-31,0.25,35816.99982
4,987390,709891,40,Рис длиннозерный 500 гр,395054,2018-01-11,14.0,2018-01-24,21,2017-12-26,2018-03-01,0.25,20898.64467
5,987395,708686,45,Рис длиннозерный 500 гр,395054,2018-01-15,14.0,2018-01-28,14,2018-01-01,2018-03-05,0.25,18636.00000
6,987396,708686,21,Рис длиннозерный 500 гр,395054,2018-01-15,14.0,2018-01-28,14,2018-01-01,2018-03-05,0.25,18636.00000
7,987397,708686,21,Рис длиннозерный 500 гр,395054,2018-01-15,14.0,2018-01-28,14,2018-01-01,2018-03-05,0.25,18636.00000
8,987401,709872,32,Рис длиннозерный 500 гр,395054,2018-01-17,7.0,2018-01-23,27,2018-01-01,2018-01-25,0.25,289615.92480
9,987406,709324,12,Рис длиннозерный 500 гр,395052,2018-01-22,21.0,2018-02-11,7,2018-01-08,2018-03-19,0.20,14400.00000


Начнем с обработки выбросов (выбросом считаем, если значение больше 2 сигм - из встречи с зак)

In [14]:
bpv_mean = sales_df["BPV"].mean()
bpv_std = sales_df["BPV"].std()
lower_bound = bpv_mean - 2 * bpv_std
upper_bound = bpv_mean + 2 * bpv_std

bpv_outliers = sales_df[(sales_df["BPV"] < lower_bound) | (sales_df["BPV"] > upper_bound)]

print(f"Outliers in BPV: {len(bpv_outliers)}")
bpv_outliers

Outliers in BPV: 54


,DFU,Customer,Period,BPV,Total Sell-in
773,Рис длиннозерный 500 гр,1,2017-05-22,161.151000,161.151000
776,Рис длиннозерный 500 гр,1,2017-06-12,117.777000,117.777000
782,Рис длиннозерный 500 гр,1,2017-07-24,199.887000,199.887000
784,Рис длиннозерный 500 гр,1,2017-08-07,604.584000,604.584000
791,Рис длиннозерный 500 гр,1,2017-09-25,351.762000,351.762000
800,Рис длиннозерный 500 гр,1,2017-11-27,89.382000,89.382000
801,Рис длиннозерный 500 гр,1,2017-12-04,272.910000,272.910000
807,Рис длиннозерный 500 гр,1,2018-01-15,304.746000,304.746000
812,Рис длиннозерный 500 гр,1,2018-02-19,93.024000,93.024000
813,Рис длиннозерный 500 гр,1,2018-02-26,179.394000,179.394000


Всего 31 выброс, это 1.5% от всех данных, можно удалить 

In [15]:
sales_df_cleaned = sales_df[(sales_df["BPV"] >= lower_bound) & (sales_df["BPV"] <= upper_bound)]

print(f"Rows after cleaning: {len(sales_df_cleaned)}")

Rows after cleaning: 2067


Посмотрим на выбросы в promo days on shelf

In [16]:
promo_days_mean = promo_df["Promo Days on shelf"].mean()
promo_days_std = promo_df["Promo Days on shelf"].std()
lower_promo_bound = promo_days_mean - 2 * promo_days_std
upper_promo_bound = promo_days_mean + 2 * promo_days_std

promo_days_outliers = promo_df[(promo_df["Promo Days on shelf"] < lower_promo_bound) | 
                               (promo_df["Promo Days on shelf"] > upper_promo_bound)]

print(f"Outliers Promo Days on shelf: {len(promo_days_outliers)}")
promo_days_outliers

Outliers Promo Days on shelf: 62


,Promo,Promo №,Customer,DFU,Promo mechanic,Start Date on shelf,Promo Days on shelf,End Date on shelf,Shipment days to promo start,First Date of shipment,End Date of shipment,"Discount, %",Units SoD
209,988648,709948,1,Рис длиннозерный 500 гр,395054,2019-11-28,37.0,2020-01-03,21,2019-11-12,2020-03-01,0.23,1120000.0
249,988902,709921,3,Рис длиннозерный 500 гр,395048,2020-01-27,40.0,2020-03-06,14,2020-01-01,2020-02-07,0.28,11028.0
283,989110,709594,41,Рис длиннозерный 500 гр,395054,2020-04-01,60.0,2020-05-30,14,2020-03-18,2020-05-05,0.20,24840.0
308,989298,709948,1,Рис длиннозерный 500 гр,395054,2020-11-26,42.0,2021-01-06,16,2020-11-10,2021-01-06,0.20,895656.0
370,989593,709005,3,Рис длиннозерный 500 гр,395048,2020-07-01,57.0,2020-08-26,14,2020-06-18,2020-07-28,0.17,3500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692,987927,709387,15,Булгур 300 гр,395054,2018-12-07,42.0,2019-01-17,16,2018-11-21,2019-02-02,0.30,11000.0
2748,988392,708616,20,Булгур 300 гр,395048,2019-08-15,38.0,2019-09-21,22,2019-07-20,2019-11-20,0.25,2052.0
2770,988572,709881,15,Булгур 300 гр,395054,2019-10-31,36.0,2019-12-05,22,2019-10-15,2020-01-31,0.25,15000.0
2841,989262,709904,40,Булгур 300 гр,395054,2020-11-30,44.0,2021-01-12,16,2020-11-14,2020-12-29,0.25,108.0


2% от общих, тоже дропнем

In [17]:
promo_df_cleaned = promo_df[(promo_df["Promo Days on shelf"] >= lower_promo_bound) & 
                            (promo_df["Promo Days on shelf"] <= upper_promo_bound)]

print(f"Rows after cleaning: {len(promo_df_cleaned)}")

Rows after cleaning: 2953


Теперь обработаем пропущенные значения в promo_df

In [18]:
missing_promo_days = promo_df_cleaned[promo_df_cleaned["Promo Days on shelf"].isna()]

missing_promo_days

,Promo,Promo №,Customer,DFU,Promo mechanic,Start Date on shelf,Promo Days on shelf,End Date on shelf,Shipment days to promo start,First Date of shipment,End Date of shipment,"Discount, %",Units SoD


а таких у нас нет

Теперь го объединим

In [11]:
promo_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2953 entries, 0 to 3016
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Promo                         2953 non-null   int64         
 1   Promo №                       2953 non-null   int64         
 2   Customer                      2953 non-null   int64         
 3   DFU                           2953 non-null   object        
 4   Promo mechanic                2953 non-null   int64         
 5   Start Date on shelf           2953 non-null   datetime64[ns]
 6   Promo Days on shelf           2953 non-null   float64       
 7   End Date on shelf             2953 non-null   datetime64[ns]
 8   Shipment days to promo start  2953 non-null   int64         
 9   First Date of shipment        2953 non-null   datetime64[ns]
 10  End Date of shipment          2953 non-null   datetime64[ns]
 11  Discount, %                   2953 

In [12]:
sales_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2067 entries, 0 to 2120
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DFU            2067 non-null   object        
 1   Customer       2067 non-null   int64         
 2   Period         2067 non-null   datetime64[ns]
 3   BPV            2067 non-null   float64       
 4   Total Sell-in  2067 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 96.9+ KB


создадим неделю для объединения

In [19]:
sales_df_cleaned['End of Period'] = sales_df_cleaned['Period'] + pd.Timedelta(days=6)

/var/folders/lm/kymf9kqj2kbfvq9cx_8cpz5h0000gn/T/ipykernel_9087/314251114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df_cleaned['End of Period'] = sales_df_cleaned['Period'] + pd.Timedelta(days=6)


In [20]:
sales_df_cleaned

,DFU,Customer,Period,BPV,Total Sell-in,End of Period
0,Рис басмати 500 гр,18,2017-07-24,0.846000,0.846000,2017-07-30
1,Рис басмати 500 гр,18,2017-10-23,1.692000,1.692000,2017-10-29
2,Рис басмати 500 гр,18,2018-02-19,2.538000,2.538000,2018-02-25
3,Рис басмати 500 гр,18,2018-04-23,1.692000,1.692000,2018-04-29
4,Рис басмати 500 гр,18,2019-04-29,4.232100,4.232100,2019-05-05
...,...,...,...,...,...,...
2116,Булгур 300 гр,1,2020-02-24,0.000000,0.032400,2020-03-01
2117,Булгур 300 гр,1,2020-03-02,0.000000,3.211500,2020-03-08
2118,Булгур 300 гр,1,2020-04-06,0.000000,0.421200,2020-04-12
2119,Булгур 300 гр,1,2020-07-06,0.008085,0.008085,2020-07-12


In [22]:
merged_df = pd.merge(
    sales_df_cleaned,
    promo_df_cleaned,
    left_on=['Customer', 'DFU'],
    right_on=['Customer', 'DFU'],
    how='left' 
)

merged_df = merged_df[
    (merged_df['Period'] <= merged_df['End Date of shipment']) &
    (merged_df['End of Period'] >= merged_df['First Date of shipment']) |
    (merged_df['First Date of shipment'].isna())
]

merged_df['is_full_period'] = (
    (merged_df['Period'] >= merged_df['First Date of shipment']) & 
    (merged_df['End of Period'] <= merged_df['End Date of shipment'])
).astype(int)

merged_df = merged_df.reset_index(drop=True)

In [23]:
merged_df

,DFU,Customer,Period,BPV,Total Sell-in,End of Period,Promo,Promo №,Promo mechanic,Start Date on shelf,Promo Days on shelf,End Date on shelf,Shipment days to promo start,First Date of shipment,End Date of shipment,"Discount, %",Units SoD,is_full_period
0,Рис басмати 500 гр,18,2017-07-24,0.8460,0.8460,2017-07-30,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,0
1,Рис басмати 500 гр,18,2017-10-23,1.6920,1.6920,2017-10-29,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,0
2,Рис басмати 500 гр,18,2018-02-19,2.5380,2.5380,2018-02-25,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,0
3,Рис басмати 500 гр,18,2018-04-23,1.6920,1.6920,2018-04-29,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,0
4,Рис басмати 500 гр,18,2019-04-29,4.2321,4.2321,2019-05-05,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,Булгур 300 гр,1,2020-02-17,0.0000,0.2184,2020-02-23,988964.0,708684.0,395054.0,2020-02-25,7.0,2020-03-02,21.0,2020-02-09,2020-02-20,0.25,2724.666667,0
1674,Булгур 300 гр,1,2020-02-17,0.0000,0.2184,2020-02-23,989037.0,709780.0,395055.0,2020-03-12,22.0,2020-04-02,21.0,2020-02-20,2020-04-03,0.25,7136.666667,0
1675,Булгур 300 гр,1,2020-02-24,0.0000,0.0324,2020-03-01,989037.0,709780.0,395055.0,2020-03-12,22.0,2020-04-02,21.0,2020-02-20,2020-04-03,0.25,7136.666667,1
1676,Булгур 300 гр,1,2020-03-02,0.0000,3.2115,2020-03-08,989037.0,709780.0,395055.0,2020-03-12,22.0,2020-04-02,21.0,2020-02-20,2020-04-03,0.25,7136.666667,1


In [24]:
merged_df['Discount, %'] = merged_df['Discount, %'] * 100

In [25]:
merged_df.to_excel("data/merged_df.xlsx")

In [28]:
merged_df['DFU'].unique()

array(['Рис басмати 500 гр', 'Рис длиннозерный 500 гр',
       'Рис круглозерный 1000 гр', 'Рис круглозерный 500 гр',
       'Рис Жасмин 500 гр', 'Рис для плова 500 гр',
       'Рис длиннозерный 486 гр', 'Гречка Ядрица 1000 гр',
       'Йогурт Постный 180 гр', 'Булгур 300 гр'], dtype=object)